# POD and fft from pointclouds 

We will now proceed to explain how to perform POD from point clouds. In this instance, we test only for POD in serial, as to perform in parallel, a parallel reader/writer is needed.

If you have saved information in hdf5 and have habilitated mpi4py compilation of it, then you could use this code in parallel.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np
import h5py
import sys

# Get mpi info
comm = MPI.COMM_WORLD

## Set up the input parameters

In [2]:
file_sequence = [f"interpolated_fields{str(1+i).zfill(5)}.hdf5" for i in range(0, 48)]
pod_fields = ["u", "v", "w"]
mesh_fname = "points.hdf5"
mass_matrix_fname = "points.hdf5"
mass_matrix_key = "mass"
k = 10
p = len(file_sequence)
p = 10
fft_axis = 1 # 0 for x, 1 for y, 2 for z

# Calculated parameters
number_of_pod_fields = len(pod_fields)

## Call the pynektools routines

In [3]:
def get_wavenumber_slice(kappa, fft_axis):
    if fft_axis == 0:
        return (kappa, slice(None), slice(None))
    elif fft_axis == 1:
        return (slice(None), kappa, slice(None))
    elif fft_axis == 2:
        return (slice(None), slice(None), kappa)

# Import IO helper functions
from pynektools.io.utils import get_fld_from_ndarray

# Import types asociated with POD
from pynektools.rom.pod import POD
from pynektools.rom.io_help import IoHelp

# Output
from pyevtk.hl import gridToVTK

# Load the mesh
with h5py.File(mesh_fname, 'r') as f:
    x = f["x"][:]
    y = f["y"][:]
    z = f["z"][:]

# Load the mass matrix
with h5py.File(mass_matrix_fname, 'r') as f:
    bm = f[mass_matrix_key][:]
bm[np.where(bm == 0)] = 1e-8

# Have a slice of the axis to perform the fft
if fft_axis == 0:
    mass_slice = (0, slice(None), slice(None))
elif fft_axis == 1:
    mass_slice = (slice(None), 0, slice(None))
elif fft_axis == 2:
    mass_slice = (slice(None), slice(None), 0)

# Obtain the number of frequencies you will obtain
N_samples = bm.shape[fft_axis]
number_of_frequencies = N_samples // 2 + 1

# Choose the proper mass matrix slice
bm = bm[mass_slice]

ioh = {"wavenumber" : "buffers"}
pod = {"wavenumber" : "POD object"}

# Initialize the buffers and objects for each wavenumber
for kappa in range(0, number_of_frequencies):

    # Instance io helper that will serve as buffer for the snapshots
    ioh[kappa] = IoHelp(comm, number_of_fields = number_of_pod_fields, batch_size = p, field_size = bm.size, mass_matrix_data_type=bm.dtype, field_data_type=np.complex128,  module_name="buffer_kappa"+str(kappa))

    # Put the mass matrix in the appropiate format (long 1d array)
    mass_list = []
    for i in range(0, number_of_pod_fields):
        mass_list.append(np.copy(np.sqrt(bm)))
    ioh[kappa].copy_fieldlist_to_xi(mass_list)
    ioh[kappa].bm1sqrt[:,:] = np.copy(ioh[kappa].xi[:,:])

    # Instance the POD object
    pod[kappa] = POD(comm, number_of_modes_to_update = k, global_updates = True, auto_expand = False)

# Perform reading and updates
j = 0
while j < len(file_sequence):

    # Load the snapshot data
    fname = file_sequence[j]
    with h5py.File(fname, 'r') as f:
        fld_data = []
        for field in pod_fields:
            fld_data.append(f[field][:])

    # Perform the fft
    for i in range(0, number_of_pod_fields):
        fld_data[i] = np.fft.fft(fld_data[i], axis = fft_axis)/N_samples

    # For each wavenumber, put the snapshot data into the buffer    
    for kappa in range(0, number_of_frequencies):

        # Get the proper slice for the wavenumber
        wavenumber_slice = get_wavenumber_slice(kappa, fft_axis)

        # Get the wavenumber data
        wavenumber_data = []
        for i in range(0, number_of_pod_fields):
            wavenumber_data.append(fld_data[i][wavenumber_slice])

        # Put the snapshot data into a column array
        ioh[kappa].copy_fieldlist_to_xi(wavenumber_data)

        # Load the column array into the buffer
        ioh[kappa].load_buffer(scale_snapshot = True)
        
        # Update POD modes
        if ioh[kappa].update_from_buffer:
            pod[kappa].update(comm, buff = ioh[kappa].buff[:,:(ioh[kappa].buffer_index)])

    j += 1

for kappa in range(0, number_of_frequencies):
    # Check if there is information in the buffer that should be taken in case the loop exit without flushing
    if ioh[kappa].buffer_index > ioh[kappa].buffer_max_index:
        ioh[kappa].log.write("info","All snapshots where properly included in the updates")
    else: 
        ioh[kappa].log.write("warning","Last loaded snapshot to buffer was: "+repr(ioh[kappa].buffer_index-1))
        ioh[kappa].log.write("warning","The buffer updates when it is full to position: "+repr(ioh[kappa].buffer_max_index))
        ioh[kappa].log.write("warning","Data must be updated now to not lose anything,  Performing an update with data in buffer ")
        pod[kappa].update(comm, buff = ioh[kappa].buff[:,:(ioh[kappa].buffer_index)])

    # Scale back the modes
    pod[kappa].scale_modes(comm, bm1sqrt = ioh[kappa].bm1sqrt, op = "div")

    # Rotate local modes back to global, This only enters in effect if global_update = false
    pod[kappa].rotate_local_modes_to_global(comm)

    print(pod[kappa].u_1t.shape)
    print(pod[kappa].u_1t.dtype)

sys.exit(0)

# Write the data in vtk for now
# Go over the modes
for j in range(0, k):
    
    ## Split the snapshots into the proper fields
    field_list1d = ioh.split_narray_to_1dfields(pod.u_1t[:,j])
    field_dict = {}
    for i in range(0, len(pod_fields)):
        field_dict[f"{pod_fields[i]}_mode"] = field_list1d[i].reshape(bm.shape)

    # write to vtk
    gridToVTK( "pod_mode"+str(j).zfill(5),  x, y, z, pointData=field_dict)

2024-09-20 13:09:40,681 - buffer_kappa0 - INFO - io_helper object initialized
2024-09-20 13:09:40,683 - pod - WARNING - ifget_all_modes is hard coded to False. This parameter applies to lcl updates. It controls if one gets all modes in the global rotation, despite keeping less modes locally. I do not see a use for this in production runs. Thus it is set to false. If needed, activate in mpi_spSVD.py module
2024-09-20 13:09:40,684 - pod - INFO - POD Object initialized
2024-09-20 13:09:40,685 - buffer_kappa1 - INFO - io_helper object initialized
2024-09-20 13:09:40,686 - pod - WARNING - ifget_all_modes is hard coded to False. This parameter applies to lcl updates. It controls if one gets all modes in the global rotation, despite keeping less modes locally. I do not see a use for this in production runs. Thus it is set to false. If needed, activate in mpi_spSVD.py module
2024-09-20 13:09:40,686 - pod - INFO - POD Object initialized
2024-09-20 13:09:40,687 - buffer_kappa2 - INFO - io_helper

/tmp/ipykernel_16502/3061128119.py:59: ComplexWarning: Casting complex values to real discards the imaginary part
  ioh[kappa].bm1sqrt[:,:] = np.copy(ioh[kappa].xi[:,:])


2024-09-20 13:09:40,884 - buffer_kappa2 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,885 - buffer_kappa3 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,886 - buffer_kappa4 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,887 - buffer_kappa5 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,888 - buffer_kappa6 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,888 - buffer_kappa7 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,890 - buffer_kappa8 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,891 - buffer_kappa9 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,892 - buffer_kappa10 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,894 - buffer_kappa11 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,895 - buffer_kappa12 - INFO - Loaded snapshot in buffer in pos: 5
2024-09-20 13:09:40,896 - buffer_kappa13 - INFO - Loaded snaps

SystemExit: 0

/home/adperez/software/miniconda3/.conda/envs/python3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
